---
title: Randomized Cholesky QR
description: Stabilized QR factorization combining sketching for preconditioning with Cholesky decomposition for efficiency
keywords: [randomized Cholesky QR, subspace embedding, sketching, preconditioning, numerical stability, well-conditioned basis]
numbering:
  equation:
    enumerator: 3.%s
    continue: true
  proof:theorem:
    enumerator: 3.%s
    continue: true
  proof:algorithm:
    enumerator: 3.%s
    continue: true
  proof:definition:
    enumerator: 3.%s
    continue: true
  proof:proposition:
    enumerator: 3.%s
    continue: true
---

While [CholeskyQR](./cholesky-qr.ipynb) has a high flop efficiency, it is unstable if $\vec{A}$ is not well-conditioned due to working with the Gram matrix $\vec{A}^\T\vec{A}$.
Fortunately, we can fix this by using a subspace embedding.


## Subspace Embedding for approximate QR


:::{prf:algorithm} Sketched QR
:label: sketched-qr

**Input:** $\vec{A}\in\R^{n\times d}$, sketching dimension $k$

1. Sample $\vec{S}\sim\Call{Sketch}(k,n)$
1. Form $\vec{Y} = \vec{S}\vec{A}$
1. Compute QR factorization $\vec{Q}'\vec{R} = \Call{qr}(\vec{Y})$.
1. Form $\vec{Q} = \vec{A}\vec{R}^{-1}$

**Output:** $\vec{Q}, \vec{R}$
:::

This algorithm is easily implemented in Numpy.
Here we use a [SparseStack](def:sparse-stack-sketch) sketch.

In [1]:
def sketched_qr(A,k,zeta,rng):

    n, d = A.shape
    S = sparse_stack_sketch(n,k,zeta,rng) 
    Y = S @ A 
    R = np.linalg.qr(Y, mode='r') 
    Q = sp.linalg.solve_triangular(R.T,A.T,lower=True).T 
    
    return Q, R

:::{prf:theorem} Well-conditioned basis from subspace embedding
:label: sketched-qr-well-conditioned

Suppose $\vec{A}$ is full-rank and $\vec{S}$ is an $\varepsilon$-subspace embedding for $\vec{A}$.
Then the output $\vec{Q}$ of {prf:ref}`sketched-qr` is well-conditioned in the sense that 
\begin{equation*}
\cond(\vec{Q}) \leq \frac{1+\varepsilon}{1-\varepsilon}.
\end{equation*}
:::


:::{admonition} Proof
:class: dropdown

By definition, 
\begin{equation*}
\forall \vec{x}\in\range(\vec{A}): (1-\varepsilon)\|\vec{x}\|_2 \leq \|\vec{S}\vec{x}\|_2 \leq (1+\varepsilon)\|\vec{x}\|_2.
\end{equation*}
We can reparameterize $\vec{x} = \vec{A}\vec{R}^{-1}\vec{c} = \vec{Q}\vec{c}$ for some $\vec{c}\in\R^d$.
Since $\vec{S}\vec{A}\vec{R}^{-1} = \vec{Q}'$ has orthonormal columns, we have $\|\vec{S}\vec{x}\| = \|\vec{S}\vec{A}\vec{R}^{-1}\vec{c}\|_2 = \|\vec{c}\|_2$.
Thus,
\begin{equation*}
(1-\varepsilon) \|\vec{Q}\vec{c}\|_2 \leq \|\vec{c}\|_2 \leq (1+\varepsilon)\|\vec{Q}\vec{c}\|_2.
\end{equation*}
Hence, 
\begin{equation*}
\smin(\vec{Q}) = \min_{\vec{c}\neq 0} \frac{\|\vec{Q}\vec{c}\|_2}{\|\vec{c}\|_2} \geq \frac{1}{1+\varepsilon} 
,\qquad
\smax(\vec{Q}) = \max_{\vec{c}\neq 0} \frac{\|\vec{Q}\vec{c}\|_2}{\|\vec{c}\|_2} \leq \frac{1}{1-\varepsilon}.
\end{equation*}
:::


Unfortunately, [oblivious sketching methods](../02-Sketching/subspace-embedding.ipynb) require $k$ to grow polynomially with $\varepsilon$. 
As such, while {prf:ref}`sketched-qr` can efficiently provide a well-conditioned basis, it is not suitable for providing a nearly machine-precision orthonormal basis.


## Preconditioning for QR


We can use the Cholesky QR method to orthonalize the output of {prf:ref}`sketched-qr`.
Since this output can bet guaranteed to be well-conditioned, the stability issues of Cholesky QR are mitigated.


```{prf:algorithm} Randomized Cholesky QR
:label: randomized-cholesky-qr

**Input:** $\vec{A}\in\R^{n\times d}$, sketching dimension $k$

1. Compute approximate QR factorization $\vec{Q}_1,\vec{R}_1 = \Call{Sketched-QR}(\vec{A})$
1. $\vec{Q},\vec{R}_1 = \Call{Cholesky-QR}(\vec{Q}_1)$
1. $\vec{R} = \vec{R}_2\vec{R}_1$

**Output:** $\vec{Q}, \vec{R}$
```

We can now use our implementation of {prf:ref}`sketched-qr` to implement {prf:ref}`randomized-cholesky-qr` in Numpy.

In [2]:
def randomized_cholesky_QR(A,k,zeta,rng):

    Q1, R1 = sketched_qr(A,k,zeta,rng)
    Q, R2 = cholesky_QR(Q1)
    R = R2 @ R1
    
    return Q, R

## Numerical Experiment

Let's compare the performance and accuracy of different QR factorization methods, including our new randomized Cholesky QR approach.
We'll use the same numerical experiment as in the previous notebook, but now we include the randomized Cholesky QR method.


In [13]:
import numpy as np
import scipy as sp
import time
import pandas as pd

import sys
sys.path.append('../')
from randnla import *

In [14]:
# Generate a random matrix with controlled condition number
n = 5000
d = 300

U, s, Vt = np.linalg.svd(np.random.rand(n, d), full_matrices=False)
s = np.geomspace(1e-4, 1, d)  # Controlled singular values for numerical stability
A = U @ np.diag(s) @ Vt


In [15]:
rng = np.random.default_rng(0) 

k = int(1.5*d)
zeta = 4

# Define QR factorization methods
qr_methods = {
    'Numpy': {
        'func': lambda: np.linalg.qr(A, mode='reduced'),
    },
    'Cholesky QR': {
        'func': lambda: cholesky_QR(A),
    },
    'Randomized Cholesky QR': {
        'func': lambda: randomized_cholesky_QR(A,k,zeta,rng),
    }
}
# Time the QR factorization methods
n_repeat = 10  # Number of repetitions for averaging

results = []

for method_name, method_info in qr_methods.items():
    # Time the method
    start = time.time()
    for _ in range(n_repeat):
        Q, R = method_info['func']()
    end = time.time()
    
    avg_time = (end - start) / n_repeat
    
    # Compute accuracy metrics
    results.append({
        'method': method_name,
        'time (s)': avg_time,
        'orthogonality': np.linalg.norm(Q.T @ Q - np.eye(d)),
        'reconstruction': np.linalg.norm(A - Q @ R)
    })

# Create DataFrame and compute relative performance
results_df = pd.DataFrame(results)
results_df['speedup'] = results_df['time (s)'] / results_df['time (s)'].max()

# Display results with formatting
results_df.reindex(columns=['method','time (s)','speedup','orthogonality','reconstruction']).style.format({
    'time (s)': '{:.4f}',
    'orthogonality': '{:1.1e}',
    'reconstruction': '{:1.1e}',
    'speedup': '{:.1f}x',
})

,method,time (s),speedup,orthogonality,reconstruction
0,Numpy,0.1148,1.0x,7.1e-15,2.9e-15
1,Cholesky QR,0.0278,0.2x,4.1e-09,6.7e-16
2,Randomized Cholesky QR,0.0596,0.5x,3.2e-14,3.8e-15


We observe that randomized Cholesky QR is able to get most of the speedups of Cholesky QR, while producing an accurate QR factorization!